In [1]:
#importing libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [2]:
#load the NSL_KDD dataset
train=pd.read_csv('dataset/KDDTrain+.csv')
test=pd.read_csv('dataset/KDDTest+.csv')

#size of train and test data
print(train.shape,test.shape)

# concatenate traina and test data
dataset=pd.concat([test,train])

#size of whole dataset
print(dataset.shape)

#print(dataset.head(2))
#print(dataset.columns)
X = dataset.iloc[:, :-1].values
print(X)
y = dataset.iloc[:, 41:42].values
print(y)

(113270, 42) (15451, 42)
(128721, 42)
[[0 'tcp' 'private' ... 0.0 1.0 1.0]
 [0 'tcp' 'private' ... 0.0 1.0 1.0]
 [2 'tcp' 'ftp_data' ... 0.0 0.0 0.0]
 ...
 [0 'tcp' 'smtp' ... 0.0 0.01 0.0]
 [0 'tcp' 'klogin' ... 1.0 0.0 0.0]
 [0 'tcp' 'ftp_data' ... 0.0 0.0 0.0]]
[['DDoS']
 ['DDoS']
 ['normal']
 ...
 ['normal']
 ['DDoS']
 ['normal']]


In [3]:
#Encoding of dataset
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1, 2, 3])], remainder = 'passthrough')
X= ct.fit_transform(X)
print(X)

[[0.0 1.0 0.0 ... 0.0 1.0 1.0]
 [0.0 1.0 0.0 ... 0.0 1.0 1.0]
 [0.0 1.0 0.0 ... 0.0 0.0 0.0]
 ...
 [0.0 1.0 0.0 ... 0.0 0.01 0.0]
 [0.0 1.0 0.0 ... 1.0 0.0 0.0]
 [0.0 1.0 0.0 ... 0.0 0.0 0.0]]


In [4]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y[:, 0] = le_y.fit_transform(y[:, 0])
y= y.astype(int)
y = y.ravel()
print(y)

[0 0 1 ... 1 0 1]


In [5]:
# min_max normalisation between (a,b)
from sklearn.preprocessing import minmax_scale
X = minmax_scale(X, feature_range=(0,1), axis=0)
print(X)

[[0.   1.   0.   ... 0.   1.   1.  ]
 [0.   1.   0.   ... 0.   1.   1.  ]
 [0.   1.   0.   ... 0.   0.   0.  ]
 ...
 [0.   1.   0.   ... 0.   0.01 0.  ]
 [0.   1.   0.   ... 1.   0.   0.  ]
 [0.   1.   0.   ... 0.   0.   0.  ]]


In [6]:
# ReliefF
from ReliefF import ReliefF
fs = ReliefF(n_neighbors=20, n_features_to_keep=50)
XRLF = fs.fit_transform(X, y)
print(XRLF)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
print(XRLF.shape)

(128721, 50)


In [8]:
# splitting of dataset in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XRLF, y, test_size = 0.3, random_state = 1)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[1 0 1 ... 1 1 1]
[1 0 0 ... 1 0 1]


In [9]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [17]:
# base classifier RandomForest
y_train = y_train.ravel()
clf1 = RandomForestClassifier(n_estimators=100)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred1)
cm.print_stats()

from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred1))
print("RMSE= ",RMSE)

population: 38617
P: 23198
N: 15419
PositiveTest: 25087
NegativeTest: 13530
TP: 22859
TN: 13191
FP: 2228
FN: 339
TPR: 0.9853866712647642
TNR: 0.8555029509047279
PPV: 0.9111890620640172
NPV: 0.9749445676274945
FPR: 0.14449704909527206
FDR: 0.08881093793598278
FNR: 0.014613328735235796
ACC: 0.9335266851386695
F1_score: 0.9468364916640779
MCC: 0.8632152530296112
informedness: 0.8408896221694921
markedness: 0.8861336296915117
prevalence: 0.6007198902037962
LRP: 6.81942418502307
LRN: 0.01708156438242747
DOR: 399.22714526302417
FOR: 0.025055432372505543
RMSE=  0.2578241937082913


In [11]:
# base classifier GradientBoosting
clf2 = GradientBoostingClassifier(n_estimators=100)
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred2)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred2))
print("RMSE= ",RMSE)

population: 38617
P: 23198
N: 15419
PositiveTest: 25481
NegativeTest: 13136
TP: 22866
TN: 12804
FP: 2615
FN: 332
TPR: 0.9856884214156393
TNR: 0.8304040469550554
PPV: 0.897374514344021
NPV: 0.9747259439707674
FPR: 0.16959595304494454
FDR: 0.10262548565597897
FNR: 0.014311578584360721
ACC: 0.9236864593313826
F1_score: 0.9394605476694262
MCC: 0.8436318010206406
informedness: 0.8160924683706947
markedness: 0.8721004583147884
prevalence: 0.6007198902037962
LRP: 5.811980791513477
LRN: 0.017234475960032643
DOR: 337.2299108479808
FOR: 0.025274056029232644
RMSE=  0.27624905550719525


In [12]:
# SVM
from sklearn.svm import SVC  
clf3= SVC(kernel='linear') 
clf3.fit(X_train, y_train) 
y_pred3 = clf3.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred3)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred3))
print("RMSE= ",RMSE)

population: 38617
P: 23198
N: 15419
PositiveTest: 25090
NegativeTest: 13527
TP: 22859
TN: 13188
FP: 2231
FN: 339
TPR: 0.9853866712647642
TNR: 0.8553083857578313
PPV: 0.9110801115982463
NPV: 0.9749390108671546
FPR: 0.14469161424216875
FDR: 0.08891988840175369
FNR: 0.014613328735235796
ACC: 0.9334489991454541
F1_score: 0.9467776673293572
MCC: 0.863059613632894
informedness: 0.8406950570225955
markedness: 0.886019122465401
prevalence: 0.6007198902037962
LRP: 6.810254183877812
LRN: 0.017085450088610912
DOR: 398.5996358631195
FOR: 0.02506098913284542
RMSE=  0.2579748066275967


In [13]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf4 = KNeighborsClassifier(n_neighbors = 5)
clf4.fit(X_train, y_train) 
y_pred4 = clf4.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred4)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred4))
print("RMSE= ",RMSE)

population: 38617
P: 23198
N: 15419
PositiveTest: 25091
NegativeTest: 13526
TP: 22860
TN: 13188
FP: 2231
FN: 338
TPR: 0.9854297784291749
TNR: 0.8553083857578313
PPV: 0.9110836554940018
NPV: 0.9750110897530682
FPR: 0.14469161424216875
FDR: 0.08891634450599817
FNR: 0.01457022157082507
ACC: 0.9334748944765259
F1_score: 0.9467994781420199
MCC: 0.8631185720483453
informedness: 0.8407381641870062
markedness: 0.88609474524707
prevalence: 0.6007198902037962
LRP: 6.810552108291998
LRN: 0.0170350505308274
DOR: 399.7964136336029
FOR: 0.024988910246931836
RMSE=  0.25792461209329


In [14]:
# meta classifier LogisticRegression
lr = LogisticRegression(solver='lbfgs')

# stack esemble method 
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf1, clf2,clf4], 
                          meta_classifier=lr)

In [15]:
# train and test the model
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)

In [16]:
from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred)
cm.print_stats()
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE= ",RMSE)

population: 38617
P: 23198
N: 15419
PositiveTest: 25091
NegativeTest: 13526
TP: 22860
TN: 13188
FP: 2231
FN: 338
TPR: 0.9854297784291749
TNR: 0.8553083857578313
PPV: 0.9110836554940018
NPV: 0.9750110897530682
FPR: 0.14469161424216875
FDR: 0.08891634450599817
FNR: 0.01457022157082507
ACC: 0.9334748944765259
F1_score: 0.9467994781420199
MCC: 0.8631185720483453
informedness: 0.8407381641870062
markedness: 0.88609474524707
prevalence: 0.6007198902037962
LRP: 6.810552108291998
LRN: 0.0170350505308274
DOR: 399.7964136336029
FOR: 0.024988910246931836
RMSE=  0.25792461209329
